In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello World: 最初の量子回路

[ベル状態](https://en.wikipedia.org/wiki/Bell_state)(2つのエンタングルした量子ビット)を構築し、3つの方法で実行します:

1. **理想的なシミュレーション** — 完璧な結果、アカウント不要
2. **ノイズありシミュレーション** — 実際のデバイスをシミュレート、アカウント不要
3. **実際の量子ハードウェア** — [IBM Quantumアカウント](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)が必要

## 回路を構築する

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## オプション1: 理想的なシミュレーション(アカウント不要)
`StatevectorSampler`を使用 — 完璧でノイズのない結果を提供するローカルシミュレーター。

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## オプション2: ノイズありシミュレーション(アカウント不要)
`FakeManilaV2`を使用 — ノイズ特性を含む実際のIBM量子デバイスを模倣するローカルシミュレーター。回路は最初にデバイスのゲートセットと量子ビット接続性にトランスパイル(適応)する必要があります。

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## オプション3: 実際の量子ハードウェア
IBM Quantumアカウントが必要です。詳細は[IBM Quantumアカウントの設定](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)をご覧ください。

このBinderセッションでまだ認証情報を保存していない場合は、まずこれを実行してください：

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**注意:** 実際のハードウェアでのジョブは、キューの状況に応じて時間がかかる場合があります。セルがまだ実行中の場合、ジョブステータスを確認し、結果を[quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me)で確認できます。

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## 次は何をする?
- **[チュートリアル](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — アルゴリズム、エラー緩和、トランスパイルなどに関するステップバイステップガイド
- **[コース](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — 量子基礎からユーティリティスケールコンピューティングまでの構造化された学習パス
- **[ローカルテストモード](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — IBM Quantumアカウントなしでほとんどのノートブックを実行